In [12]:
import pandas as pd
import datetime as dt
import re
from datetime import timedelta
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
import warnings
import heapq
from math import inf
warnings.filterwarnings('ignore')


# RUN BOTH OF THE ABOVE ONLY ONCE!

In [2]:
# IMPORTANT!!! READ ME
# CHANGE DATALEN FOR MORE ENTRIES. WILL EFFECT PERFORMANCE DURING TRAINING. -SAM

datalen = 60000




if datalen > 140000:
    datalen = 140000
pd.set_option('display.max_columns', None)
data = pd.read_csv('B:/incident_data.csv')
print(data)

            number incident_state  active  reassignment_count  reopen_count  \
0       INC0000045            New    True                   0             0   
1       INC0000045       Resolved    True                   0             0   
2       INC0000045       Resolved    True                   0             0   
3       INC0000045         Closed   False                   0             0   
4       INC0000047            New    True                   0             0   
...            ...            ...     ...                 ...           ...   
141707  INC0120835         Closed   False                   1             0   
141708  INC0121064         Active    True                   0             0   
141709  INC0121064         Active    True                   1             0   
141710  INC0121064       Resolved    True                   1             0   
141711  INC0121064         Closed   False                   1             0   

        sys_mod_count  made_sla    caller_id       

In [3]:



data = data[:datalen]
data.replace(['?'],'Unknown',inplace=True)
Time = data.loc[:,['opened_at','closed_at','resolved_at']]
opened = Time['opened_at']
resolved = Time['resolved_at']
closed = Time['closed_at']
X = data.loc[:,['contact_type','impact','urgency','priority','notify']]
X['open_date'] = [0] * datalen
X['total_time'] = [0] * datalen
X['caseclose_time'] = [0] * datalen

for i in range(0,len(X)):
    temp1 = str(opened[i])
    temp1 = temp1.split(" ")
    temp1[0] = temp1[0].split("/")
    temp1[1] = temp1[1].split(":")
    datetime1 = dt.datetime(int(temp1[0][2]),int(temp1[0][1]),int(temp1[0][0]),int(temp1[1][0]),int(temp1[1][1]))

    hour = temp1[1][0]
    if len(hour) < 2:
        hour = f"0{hour}"
    X['open_date'][i] = float(f"{hour}.{temp1[1][1]}")
    
    
    temp2 = str(resolved[i])
    if temp2 == 'Unknown':
        temp2 = str(opened[i])
    temp2 = temp2.split(" ")
    temp2[0] = temp2[0].split("/")
    temp2[1] = temp2[1].split(":")
    datetime2 = dt.datetime(int(temp2[0][2]),int(temp2[0][1]),int(temp2[0][0]),int(temp2[1][0]),int(temp2[1][1]))

    temp3 = str(closed[i])
    if (temp3 == 'Unknown'):
        temp3 = str(opened[i])
    temp3 = temp3.split(" ")
    temp3[0] = temp3[0].split("/")
    temp3[1] = temp3[1].split(":")
    datetime3 = dt.datetime(int(temp3[0][2]), int(temp3[0][1]), int(temp3[0][0]), int(temp3[1][0]), int(temp3[1][1]))

    totalSeconds = (datetime2-datetime1).total_seconds()
    if totalSeconds < 0:
        totalSeconds = 0
    X['total_time'][i] = totalSeconds
    totalSeconds2 = (datetime3 - datetime1).total_seconds()
    if totalSeconds2 < 0:
        totalSeconds2 = 0
    X['caseclose_time'][i] = (datetime3 - datetime1).total_seconds()

'''XCopy = X
print("Contact Type:")
XCopy['contact_type'] = XCopy['contact_type'].astype('category')
print(XCopy['contact_type'].cat.categories)

print("Impact:")
XCopy['impact'] = XCopy['impact'].astype('category')
print(XCopy['impact'].cat.categories)

print("Urgency:")
XCopy['urgency'] = XCopy['urgency'].astype('category')
print(XCopy['urgency'].cat.categories)

print("Priority:")
XCopy['priority'] = XCopy['priority'].astype('category')
print(XCopy['priority'].cat.categories)

print("Notify:")
XCopy['notify'] = XCopy['notify'].astype('category')
print(XCopy['notify'].cat.categories)'''



# START ACTUAL DATA COLLECTION BELOW 

Y = X['total_time']
Y2 = X['caseclose_time']
X2 = X


enc= LabelEncoder()
X = X.loc[:,['contact_type','impact','urgency','open_date']]
for i in (0,1,2):
    X.iloc[:,i] = enc.fit_transform(X.iloc[:,i])
X_train, X_temp, Y_train, Y_temp = train_test_split(X,Y,test_size=0.8,random_state=1)
X_valid, X_test, Y_valid, Y_test = train_test_split(X,Y,test_size=0.5,shuffle = True)
model = LinearRegression()
model.fit(X_train, Y_train)
Y_pred = model.predict(X_test)


X2 = X2.loc[:,['contact_type','impact','urgency','open_date']]
for i in (0,1,2):
    X2.iloc[:,i] = enc.fit_transform(X2.iloc[:,i])
X2_train, X2_temp, Y2_train, Y2_temp = train_test_split(X2,Y2,test_size=0.8,random_state=1)
X2_valid, X2_test, Y2_valid, Y2_test = train_test_split(X2,Y2,test_size=0.5,shuffle = False)
model2 = LinearRegression()
model2.fit(X2_train, Y2_train)
Y2_pred = model2.predict(X2_test)

print("Model has been trained.")




Model has been trained.


# RUN THE ABOVE CODE ONLY ONCE ^^^
# RUN THE BELOW PROGRAM TO TEST!

In [53]:
graph = {'police':{'a':2,'c':3},
         'a':{'police':2,'b':4,'c':3,'f':2},
         'b':{'a':4,'c':1,'d':2},
         'c':{'police':3,'a':3,'b':4,'d':8,'e':2,'g':3},
         'd':{'c':3,'e':7},
         'e':{'c':2,'d':9,'f':3},
         'f':{'a':2,'e':3,'g':2},
         'g':{'f':2,'c':3}}


def dijkstraAlg(graph,start,goal):
    shortestDist = {}
    prev = {}
    graphDupe = graph
    path = []
    for node in graphDupe:
        shortestDist[node] = inf
    shortestDist[start] = 0

    while graphDupe:
        minNode = None
        for node in graphDupe:
            if minNode is None:
                minNode = node
            elif shortestDist[node] < shortestDist[minNode]:
                minNode = node
        for edge, weight in graph[minNode].items():
            if weight + shortestDist[minNode] < shortestDist[edge]:
                shortestDist[edge] = weight + shortestDist[minNode]
                prev[edge] = minNode
        graphDupe.pop(minNode)

    currentNode = goal
    while currentNode != start:
        try:
            path.insert(0,currentNode)
            currentNode = prev[currentNode]
        except KeyError:
            print('Path not reachable')
            break
    path.insert(0,start)
    return shortestDist[goal]


graphdata = ["a","b","c","d","e","f"]

contact = input("What is your CONTACT TYPE? (Email or Phone.)   ANSWER:")
contact = contact.lower()
while(contact != "email" and contact != "phone"):
    print(contact)
    contact = input("What is your contact type? (Email or Phone.)   ANSWER:")
    contact = contact.lower()
if(contact == "phone"):
    contact = 0
else:
    contact = 1

impact = input("What is the IMPACT of your call? How many people are effected? (Low, Medium, High)   ANSWER:")
impact = impact.lower()
while(impact != "low" and impact != "medium" and impact != "high"):
    print(impact)
    impact = input("What is the IMPACT of your call? How many people are effected? (Low, Medium, High)   ANSWER:")
    impact = impact.lower()
if(impact == "high"):
    impact = 1
elif(impact == "medium"):
    impact = 2
else:
    impact = 3

urgency = input("How URGENT is your call? Life threatening, ETC. (Low, Medium, High)   ANSWER:")
urgency = urgency.lower()
while(urgency != "low" and urgency != "medium" and urgency != "high"):
    print(urgency)
    urgency = input("What is the IMPACT of your call? How many people are effected? (Low, Medium, High)   ANSWER:")
    urgency = urgency.lower()
if(urgency == "high"):
    urgency = 1
elif(urgency == "medium"):
    urgency = 2
else:
    urgency = 3

def is_valid_time(s):
    return bool(re.match(r"^(?:[01]\d|2[0-3]):[0-5]\d$", s))


date = input("What is the time of your call? Format as XX:XX in Military time. Use leading 0s.   ANSWER:")
while(not is_valid_time(date)):
    date = input("What is the time of your call? Format as XX:XX in Military time. Use leading 0s.   ANSWER:")

date = date.split(":")

date = float(f"{date[0]}.{date[1]}")

location = input("What is your location? (A, B ,C, D, E, or F)")
location =  location.lower()
while (location not in graphdata):
    location = input("What is your location? (A, B ,C, D, E, or F)")
    location =  location.lower()
        
distance = dijkstraAlg(graph,'police',location)

X_Actual =  ({'contact_type':[contact], 'impact':[impact], 'urgency':[urgency], 'open_date':[date]})
df = pd.DataFrame(X_Actual)

DFTest1 = df.loc[:,['contact_type','impact','urgency','open_date']]
Y_pred = model.predict(DFTest1)
Y_pred2 = model2.predict(DFTest1)
print("RESULTS:  \n\n")
print(f"Time until police arrival: {timedelta(seconds=round(int(Y_pred**0.52) * (distance/6)))}")
print(f"Time until case is OFFICIALLY closed in Police database: {timedelta(seconds=int(Y_pred2**0.9))}")


What is your CONTACT TYPE? (Email or Phone.)   ANSWER: Phone
What is the IMPACT of your call? How many people are effected? (Low, Medium, High)   ANSWER: High
How URGENT is your call? Life threatening, ETC. (Low, Medium, High)   ANSWER: High
What is the time of your call? Format as XX:XX in Military time. Use leading 0s.   ANSWER: 12:30
What is your location? (A, B ,C, D, E, or F) A


RESULTS:  


Time until police arrival: 0:06:02
Time until case is OFFICIALLY closed in Police database: 3 days, 10:12:37
